In [18]:
import os
import re
import glob
import json
from docx import Document
from jiwer import wer, cer, Compose, ToLowerCase, RemovePunctuation, RemoveMultipleSpaces, Strip
import pandas as pd

In [43]:
json_dir = '/Users/forcemajor01/data_science/work_place/butterboard/butterboard_interview_analysis/speech_recognition_validation/results/filtered/'
docx_dir = '/Users/forcemajor01/data_science/work_place/butterboard/butterboard_interview_analysis/results/postprocessed/'

# List JSON and DOCX files sorted by index
json_files = sorted(glob.glob(json_dir + 'recognition_result_*_interview_filtered.json'))
docx_files = sorted(glob.glob(docx_dir + 'interview_syst_analyst_*_cleaned_final.docx'))

print(f"Found {len(json_files)} JSON files and {len(docx_files)} DOCX files.")


Found 8 JSON files and 8 DOCX files.


In [46]:
JSON_PATTERN = r"recognition_result_(\d+)_interview_filtered\.json$"
DOCX_PATTERN = r"interview_syst_analyst_(\d+)_cleaned_final\.docx$"


In [47]:
NORMALIZER = Compose([
    ToLowerCase(),
    RemovePunctuation(),
    RemoveMultipleSpaces(),
    Strip()
])

In [48]:
def load_json_transcript(path):
    data = json.load(open(path, encoding='utf-8'))
    segments = sorted(data, key=lambda x: int(x.get('time_ms', 0)))
    texts = []
    for seg in segments:
        txt = seg.get('text')
        if txt:
            texts.append(txt)
    return ' '.join(texts)

def load_docx_transcript(path):
    doc = Document(path)
    texts = []
    for para in doc.paragraphs:
        line = para.text.strip()
        if not line: continue
        parts = line.split(' - ', 2)
        text = parts[2] if len(parts)==3 else line
        texts.append(text)
    return ' '.join(texts)

def normalize(text):
    return NORMALIZER(text)

In [49]:
def find_files(directory, pattern):
    files = glob.glob(os.path.join(directory, '*'))
    mapping = {}
    for f in files:
        m = re.search(pattern, os.path.basename(f))
        if m:
            mapping[int(m.group(1))] = f
    return mapping

In [50]:
json_map = find_files(json_dir, JSON_PATTERN)
docx_map = find_files(docx_dir, DOCX_PATTERN)
common = sorted(set(json_map.keys()) & set(docx_map.keys()))
common

[1, 2, 3, 4, 5, 6, 7, 8]

In [51]:
normalize(load_json_transcript(json_map[1]))

'что такое системный анализ в этом есть человек который бизнес анализом занимается значит видимо ну главным образом бизнес аналитик он занимается такими проблемами и более не бизнеса то есть он общается с общается с с заказчиком и переводит этого язык который необходим уже в дальнейшем мне то есть например есть задача там повысить посещаемость площадок на каких то на 20 он это переводит в уже более наш язык язык как это можно сделать и он больше коммуницирует с людьми получается и с тейк олдерами а я уже занимаюсь если на роли системного аналитика то превращаю эту такую более общую информацию в спецификации в задаче для разработки вот то есть система аналитика больше про управление своим проектом а не по общению с бизнесом вот то есть все равно аналитика больше про управление своим проектом а не по общению с бизнесом так хорошо тогда какой вопрос вот сайт который вы разрабатываете внутренний фильм не квартал это проект или продукт так хорошо тогда какой вопрос вот сайт который вы разра

In [52]:
normalize(load_docx_transcript(docx_map[1]))

'что такое системный анализ ты же говорил что у тебя там есть человек который бизнесанализом занимается значит видимо ты представляешь в чём суть бизнесанализа в чём отличие системного анализа от бизнесанализа ну главным образом бизнесаналитик он занимается такими проблемами и болями бизнеса общаться с получается заказчикам и переводит это в язык который необходим уже в дальнейшем мне то есть например есть задача там повысить посещаемость площадок какихто на 20 он это переводит в уже более наш язык как это можно сделать и он больше коммуницирует с людьми получается и с тейкхолдерами а я уже занимаюсь если на роли системного аналитика то превращаю эту такую более общую информацию в спецификации в задачи для разработки вот то есть системный аналитик это больше про уже управление самим проектом а не про общение с бизнесом так хорошо а так я когда вопрос вот сайт который вы разрабатываете внутренний внутренний портал это всётаки проект или продукт наверное больше проект почему потому что у

In [55]:
rows = []
for idx in common:
    ref = normalize(load_json_transcript(json_map[idx]))
    hyp = normalize(load_docx_transcript(docx_map[idx]))
    rows.append({
        'index': idx,
        'json_file': os.path.basename(json_map[idx]),
        'docx_file': os.path.basename(docx_map[idx]),
        'wer': wer(ref, hyp),
        'cer': cer(ref, hyp)
    })

df = pd.DataFrame(rows).sort_values('index').reset_index(drop=True)
df['wer_pct'] = df['wer'] * 100
df['cer_pct'] = df['cer'] * 100

print(df['wer_pct'].mean())
print(df['cer_pct'].mean())



31.228228477937055
19.610487832067296


In [56]:
df

,index,json_file,docx_file,wer,cer,wer_pct,cer_pct
0,1,recognition_result_1_interview_filtered.json,interview_syst_analyst_1_cleaned_final.docx,0.529744,0.407766,52.974398,40.776576
1,2,recognition_result_2_interview_filtered.json,interview_syst_analyst_2_cleaned_final.docx,0.226518,0.109353,22.651767,10.935252
2,3,recognition_result_3_interview_filtered.json,interview_syst_analyst_3_cleaned_final.docx,0.253495,0.115739,25.349544,11.573936
3,4,recognition_result_4_interview_filtered.json,interview_syst_analyst_4_cleaned_final.docx,0.230621,0.119765,23.062096,11.976536
4,5,recognition_result_5_interview_filtered.json,interview_syst_analyst_5_cleaned_final.docx,0.250171,0.133397,25.017112,13.339731
5,6,recognition_result_6_interview_filtered.json,interview_syst_analyst_6_cleaned_final.docx,0.232107,0.103516,23.210664,10.351628
6,7,recognition_result_7_interview_filtered.json,interview_syst_analyst_7_cleaned_final.docx,0.506848,0.432362,50.684818,43.236171
7,8,recognition_result_8_interview_filtered.json,interview_syst_analyst_8_cleaned_final.docx,0.268754,0.146941,26.875430,14.694072
